In [18]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import hilbert
from antropy import sample_entropy
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import mode
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler



In [19]:
bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45)
}

def bandpower(sig, fs, band):  #filtrira signale po frekvencijama
    low, high = band
    b, a = butter(4, [low / (fs / 2), high / (fs / 2)], btype='band')
    filtered = filtfilt(b, a, sig)
    power = np.sum(filtered ** 2) / len(filtered)
    return power 

def compute_esis(epoch, fs):
    
    energy = np.square(epoch)
    slope = np.abs(np.diff(epoch, prepend=epoch[0]))
    v = fs * slope
    esis = np.sum(energy * v)

    return esis

In [20]:
def compute_features(epoch, fs = 100):
    rms = np.sqrt(np.mean(np.square(epoch))) #ROOT MEAN SQUARE
    mse = np.mean((epoch - np.mean(epoch)) ** 2) #MEAN SQUARED ERROR
    mmd = np.mean(np.abs(np.diff(epoch))) #MINIMAL MAXIMAL DISTANCE
    zcr = ((epoch[:-1] * epoch[1:]) < 0).sum() #ZERO CROSSING RATE
    se = sample_entropy(epoch) #SAMPLE ENTROPY
    esis = compute_esis(epoch, fs)

    talasi = {ime: bandpower(epoch, fs, opseg) for ime, opseg in bands.items()}

    red = {
        "RMS": rms,
        "MSE": mse,
        "MMD": mmd,
        "ZCR": zcr,
        "Sample Entropy": se,
        "Esis": esis,
        "Delta": talasi["delta"],
        "Theta": talasi["theta"],
        "Alpha": talasi["alpha"],
        "Beta": talasi["beta"],
        "Gamma": talasi["gamma"]
    }

    return [rms, mse, mmd, zcr, se, esis, talasi["alpha"], talasi["beta"], talasi["gamma"], talasi["delta"], talasi["theta"]]

In [21]:
folder = "C:/Analiza Faza Sna/"
files = [f for f in os.listdir(folder) if f.endswith(".csv")]

all_subjects = []

for file in files:
    df = pd.read_csv(os.path.join(folder, file))
    df = df[df["Sleep_Stage"] != "P"].copy()

    signal = df["F4-M1"].values
    labels = df["Sleep_Stage"].values
    fs = 100
    samples_per_epoch = fs * 30
    num_epochs = len(signal) // samples_per_epoch

    signal_epochs = np.array_split(signal[:num_epochs * samples_per_epoch], num_epochs)
    label_epochs = np.array_split(labels[:num_epochs * samples_per_epoch], num_epochs)

    normalized_epochs = []

    for epoch in signal_epochs:
        mean = np.mean(epoch)
        std = np.std(epoch)
        if std != 0:
            norm_epoch = (epoch - mean) / std
        else:
            norm_epoch = epoch - mean
        normalized_epochs.append(norm_epoch)

    epoch_labels = [pd.Series(epoch).mode()[0] for epoch in label_epochs]
    features = [compute_features(epoch) for epoch in normalized_epochs]
    all_subjects.append((np.array(features), np.array(epoch_labels)))


In [ ]:
le = LabelEncoder()

reports = []

for i in range(len(all_subjects)):
    X_test, y_test = all_subjects[i]
    X_train = np.vstack([all_subjects[j][0] for j in range(len(all_subjects)) if j != i])
    y_train = np.concatenate([all_subjects[j][1] for j in range(len(all_subjects)) if j != i])

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test kkkk)

    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)

    nn = MLPClassifier(hidden_layer_sizes=(250, 100, 25), activation='logistic', solver='adam', max_iter=1000, random_state=42)
    nn.fit(X_train, y_train_enc)

    y_pred = nn.predict(X_test)

    print(f"\nSubject {i + 1} - {files[i]}")
    unique_labels = np.unique(np.concatenate([y_test_enc, y_pred]))
    unique_labels = np.unique(np.concatenate([y_test_enc, y_pred]))
    print(classification_report(
        y_test_enc, 
        y_pred, 
        labels=unique_labels,
        target_names=le.inverse_transform(unique_labels)
    ))





Subject 1 - S002_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.08      0.06      0.07        64
          N2       0.67      0.91      0.77       334
          N3       0.00      0.00      0.00         0
           R       0.30      0.14      0.19        80
           W       0.83      0.61      0.70       264

    accuracy                           0.65       742
   macro avg       0.38      0.34      0.35       742
weighted avg       0.64      0.65      0.62       742



c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 2 - S003_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.14      0.34      0.20        32
          N2       0.74      0.70      0.72       428
          N3       1.00      0.71      0.83       136
           R       0.46      0.47      0.46       119
           W       0.36      0.40      0.38       113

    accuracy                           0.61       828
   macro avg       0.54      0.52      0.52       828
weighted avg       0.66      0.61      0.63       828


Subject 3 - S004_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.47      0.25      0.32       163
          N2       0.71      0.75      0.73       423
          N3       0.00      0.00      0.00         1
           R       0.00      0.00      0.00         0
           W       0.46      0.31      0.37       248

    accuracy                           0.52       835
   macro avg       0.33      0.26      0.28       835
wei

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 4 - S005_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.00      0.00      0.00        39
          N2       0.83      0.58      0.68       479
          N3       0.02      0.40      0.04         5
           R       0.55      0.28      0.38       116
           W       0.35      0.70      0.46        93

    accuracy                           0.52       732
   macro avg       0.35      0.39      0.31       732
weighted avg       0.67      0.52      0.56       732


Subject 5 - S006_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.27      0.18      0.22       122
          N2       0.45      0.71      0.55       290
          N3       1.00      0.11      0.20       178
           R       0.00      0.00      0.00         0
           W       0.59      0.32      0.42       252

    accuracy                           0.39       842
   macro avg       0.46      0.27      0.28       842
wei

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 6 - S007_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.00      0.00      0.00        26
          N2       0.71      0.91      0.80       299
          N3       0.50      0.35      0.41        26
           R       0.09      0.37      0.14        41
           W       0.95      0.44      0.60       428

    accuracy                           0.59       820
   macro avg       0.45      0.41      0.39       820
weighted avg       0.78      0.59      0.63       820


Subject 7 - S008_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.16      0.12      0.14        42
          N2       0.86      0.81      0.84       421
          N3       0.55      0.63      0.59        89
           R       0.86      0.63      0.73       137
           W       0.45      0.91      0.61        58

    accuracy                           0.73       747
   macro avg       0.58      0.62      0.58       747
wei

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 10 - S011_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.16      0.17      0.16        30
          N2       0.69      0.87      0.77       423
          N3       0.64      0.05      0.09       140
           R       0.67      0.46      0.54       175
           W       0.29      0.78      0.42        45

    accuracy                           0.61       813
   macro avg       0.49      0.46      0.40       813
weighted avg       0.64      0.61      0.56       813


Subject 11 - S012_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.28      0.06      0.09       123
          N2       0.80      0.86      0.83       435
          N3       0.00      0.00      0.00         0
           R       0.65      0.94      0.77       124
           W       0.66      0.63      0.65       172

    accuracy                           0.71       854
   macro avg       0.48      0.50      0.47       854
w

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri